In [8]:
# Import required libraries  
import os  
from openai import OpenAI
from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient 
from azure.search.documents.models import (
    VectorizedQuery,
    VectorFilterMode,    
)
from backend.tools.searchtool import Search
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
credential = AzureKeyCredential(key)
openai_client = OpenAI()
openai_model = os.getenv("OPENAI_MODEL")

search_client: Search = Search('sales_vector_index')  # get instance of search to query corpus using the name of the index

creating search client with - sales_vector_index


In [9]:
def get_embedding(text, model='text-embedding-ada-002'):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

In [10]:
# Pure Vector Search
query = "describe challenger sales model"  
  
results = search_client.search_hybrid(query)  
  

Starting Search - describe challenger sales model
Got embedding
Got result


HttpResponseError: (InvalidRequestParameter) The selected filter mode 'preFilter' is not supported by this index. Only new indexes opted in will support the filter type.
Parameter name: vector.filterMode
Code: InvalidRequestParameter
Message: The selected filter mode 'preFilter' is not supported by this index. Only new indexes opted in will support the filter type.
Parameter name: vector.filterMode
Exception Details:	(InvalidVectorQuery) The selected filter mode 'preFilter' is not supported by this index. Only new indexes opted in will support the filter type.
	Code: InvalidVectorQuery
	Message: The selected filter mode 'preFilter' is not supported by this index. Only new indexes opted in will support the filter type.

In [ ]:
# Hybrid
query = "bedrock"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
  
vector_query = VectorizedQuery(vector=get_embedding(query), k_nearest_neighbors=3, fields="contentVector")
  
category = "AWS"

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter=f"category eq '{category}'",
    #filter="category eq 'Microsoft'",
    select=["title", "content", "category", "spr"],
)
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  
    print(f"SPR: {result['spr']}\n")  